# Indirect Object Identification Circuit in Pythia

## Imports

In [1]:
import os

import torch
from torch import Tensor
import numpy as np
import einops
from fancy_einsum import einsum
# import circuitsvis as cv

from transformers import AutoModelForCausalLM, AutoTokenizer

import transformer_lens.utils as tl_utils
from transformer_lens import HookedTransformer
import transformer_lens.patching as patching

from jaxtyping import Float
import plotly.express as px

from functools import partial

from torchtyping import TensorType as TT

from path_patching_cm.path_patching import Node, IterNode, path_patch, act_patch
from path_patching_cm.ioi_dataset import IOIDataset, NAMES
from neel_plotly import imshow as imshow_n

from utils.data_utils import UniversalPatchingDataset
from utils.metrics import compute_logit_diff



if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(tl_utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=tl_utils.to_numpy(tensor), **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[tl_utils.to_numpy(tensor1), tl_utils.to_numpy(tensor2)], **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = tl_utils.to_numpy(x)
    y = tl_utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Parameters

In [3]:
BASE_MODEL = "pythia-160m"
VARIANT = None
CACHE = "model_cache"
CHECKPOINT = 143000
torch.set_grad_enabled(False)

IOI_DATASET_SIZE = 70

## Model Setup

In [4]:
def get_model(base_model="pythia-160m", variant=None, checkpoint=143000, cache="model_cache"):
    if not variant:
        model = HookedTransformer.from_pretrained(
            base_model,
            checkpoint_value=checkpoint,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False,
            dtype=torch.bfloat16,
            **{"cache_dir": cache},
        )
    else:
        revision = f"step{checkpoint}"
        source_model = AutoModelForCausalLM.from_pretrained(
        VARIANT, revision=revision, cache_dir=cache
        ).to(device).to(torch.bfloat16)

        model = HookedTransformer.from_pretrained(
            base_model,
            hf_model=source_model,
            center_unembed=False,
            center_writing_weights=False,
            fold_ln=False,
            dtype=torch.bfloat16,
            **{"cache_dir": cache},
        )
        
    model.cfg.use_split_qkv_input = True
    model.cfg.use_attn_result = True
    model.cfg.use_hook_mlp_in = True

    return model

## Data Setup

In [5]:
model = get_model()
ds = UniversalPatchingDataset.from_ioi(model, IOI_DATASET_SIZE)
# logit_diff_metric = partial(compute_logit_diff,mode='simple')

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


In [6]:
clean_logits, clean_cache = model.run_with_cache(ds.toks)
# corrupted_logits, corrupted_cache = model.run_with_cache(ds.flipped_toks)

# clean_logit_diff = logit_diff_metric(clean_logits, ds.answer_toks, ds.positions)
# print(f"Clean logit diff: {clean_logit_diff:.4f}")

# corrupted_logit_diff = logit_diff_metric(corrupted_logits, ds.answer_toks, ds.positions)
# print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")

In [7]:
clean_cache

ActivationCache with keys ['hook_embed', 'blocks.0.hook_resid_pre', 'blocks.0.hook_q_input', 'blocks.0.hook_k_input', 'blocks.0.hook_v_input', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_rot_q', 'blocks.0.attn.hook_rot_k', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.hook_result', 'blocks.0.hook_attn_out', 'blocks.0.hook_mlp_in', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.hook_q_input', 'blocks.1.hook_k_input', 'blocks.1.hook_v_input', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_rot_q', 'blocks.1.attn.hook_rot_k', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.ho

In [8]:
layer_idx = 8
head_idx = 9
clean_cache[f"blocks.{layer_idx}.attn.hook_z"][:, :, head_idx, :].shape

torch.Size([70, 21, 64])

In [9]:
clean_cache[f"blocks.{layer_idx}.attn.hook_pattern"][:, head_idx, :, :].shape

torch.Size([70, 21, 21])

In [16]:
clean_cache["blocks.8.attn.hook_pattern"][0, 9, :, 18]

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.3612e-06, 7.8669e-05, 2.0254e-05], device='cuda:0')

In [13]:
ds.positions[0]

tensor(18)

In [17]:
from transformer_lens import ActivationCache

ActivationCache?

Init signature:
ActivationCache(
    cache_dict: 'Dict[str, torch.Tensor]',
    model,
    has_batch_dim: 'bool' = True,
)
Docstring:     
Activation Cache.

A wrapper that stores all important activations from a forward pass of the model, and provides a
variety of helper functions to investigate them.

The :class:`ActivationCache` is at the core of Transformer Lens. It is a wrapper that stores all
important activations from a forward pass of the model, and provides a variety of helper
functions to investigate them. The common way to access it is to run the model with
:meth:`transformer_lens.HookedTransformer.run_with_cache`.

Examples:

When investigating a particular behaviour of a modal, a very common first step is to try and
understand which components of the model are most responsible for that behaviour. For example,
if you're investigating the prompt "Why did the chicken cross the" -> " road", you might want to
understand if there is a specific sublayer (mlp or multi-head attenti

## Attention Head Comparison

In [18]:
def get_components(cache, positions, layer_idx, head_idx):
    return {
        "act_Q": cache[f"blocks.{layer_idx}.attn.hook_q"][:, positions, head_idx, :].detach().to("cpu"),
        "act_K": cache[f"blocks.{layer_idx}.attn.hook_k"][:, positions, head_idx, :].detach().to("cpu"),
        "act_V": cache[f"blocks.{layer_idx}.attn.hook_v"][:, positions, head_idx, :].detach().to("cpu"),
        "act_Z": cache[f"blocks.{layer_idx}.attn.hook_z"][:, positions, head_idx, :].detach().to("cpu"),
        "attn_pattern": cache[f"blocks.{layer_idx}.attn.hook_pattern"][:, head_idx, positions, :].detach().to("cpu"),
    }

In [19]:
def component_cosine_sim(model_a, model_b, heads, dataset=ds):
    _, cache_a = model_a.run_with_cache(dataset.toks)
    _, cache_b = model_b.run_with_cache(dataset.toks)

    cosine_sims = dict()

    for layer_idx, head_idx in heads:

        cosine_sims[f"L{layer_idx}H{head_idx}"] = dict()

        components_a = get_components(cache_a, dataset.positions, layer_idx, head_idx)
        components_b = get_components(cache_b, dataset.positions, layer_idx, head_idx)

        for k in components_a.keys():
            cosine_sims[f"L{layer_idx}H{head_idx}"][k] = torch.nn.functional.cosine_similarity(components_a[k].flatten(), components_b[k].flatten(), dim=0).item()

    return cosine_sims

In [22]:
model_a = get_model(checkpoint=10000)
model_b = get_model(checkpoint=5000)

cosine_sims = component_cosine_sim(model_a, model_b, [(8, 9)])
cosine_sims

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


{'L8H9': {'act_Q': 0.8984375,
  'act_K': 0.97265625,
  'act_V': 0.8046875,
  'act_Z': 0.8046875,
  'attn_pattern': 0.9296578168869019}}

### Get Top Heads

In [23]:
import json

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

model_a = get_model(checkpoint=143000)

similarity_through_time = dict()

heads = [(8, 9), (10, 1), (4, 11), (6, 6), (7, 2), (2, 6), (6, 5), (4, 6), (10, 7), (8, 2), (8, 10), (7, 9), (4, 1), (4, 8), (5, 8), (9, 8),
         (10, 11), (9, 6), (6, 11), (5, 0), (3, 0), (11, 6)]

for c in range(1000, 142000, 1000):
    
    model_b = get_model(checkpoint=c)

    cosine_sims = component_cosine_sim(model_a, model_b, heads)
    similarity_through_time[c] = cosine_sims

    print(f"Checkpoint: {c} - Head Component Cosine Similarity to 143K: {cosine_sims}")

    # save json
    with open("similarity_through_time_last_pos.json", "w") as f:
        json.dump(similarity_through_time, f)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


Checkpoint: 1000 - Head Component Cosine Similarity to 143K: {'L8H9': {'act_Q': 0.2294921875, 'act_K': 0.201171875, 'act_V': 0.015869140625, 'act_Z': -0.0228271484375, 'attn_pattern': 0.28632456064224243}, 'L10H1': {'act_Q': 0.1318359375, 'act_K': 0.1904296875, 'act_V': -0.02392578125, 'act_Z': -0.0177001953125, 'attn_pattern': 0.14213140308856964}, 'L4H11': {'act_Q': 0.62109375, 'act_K': 0.328125, 'act_V': 0.15625, 'act_Z': 0.140625, 'attn_pattern': 0.5096052885055542}, 'L6H6': {'act_Q': 0.412109375, 'act_K': 0.1962890625, 'act_V': 0.181640625, 'act_Z': 0.00885009765625, 'attn_pattern': 0.33000850677490234}, 'L7H2': {'act_Q': -0.09521484375, 'act_K': 0.123046875, 'act_V': 0.0966796875, 'act_Z': 0.07080078125, 'attn_pattern': 0.4469740390777588}, 'L2H6': {'act_Q': 0.62109375, 'act_K': 0.42578125, 'act_V': 0.296875, 'act_Z': 0.16015625, 'attn_pattern': 0.6997661590576172}, 'L6H5': {'act_Q': 0.1298828125, 'act_K': 0.07470703125, 'act_V': 0.08984375, 'act_Z': 0.025390625, 'attn_pattern': 

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


Checkpoint: 140000 - Head Component Cosine Similarity to 143K: {'L8H9': {'act_Q': 0.97265625, 'act_K': 1.0, 'act_V': 0.97265625, 'act_Z': 0.90625, 'attn_pattern': 0.9214602708816528}, 'L10H1': {'act_Q': 0.97265625, 'act_K': 0.99609375, 'act_V': 0.85546875, 'act_Z': 0.734375, 'attn_pattern': 0.8316106796264648}, 'L4H11': {'act_Q': 1.0, 'act_K': 1.0, 'act_V': 0.98046875, 'act_Z': 0.98046875, 'attn_pattern': 0.9956150054931641}, 'L6H6': {'act_Q': 0.98828125, 'act_K': 1.0, 'act_V': 0.98046875, 'act_Z': 0.9765625, 'attn_pattern': 0.9952161908149719}, 'L7H2': {'act_Q': 0.98046875, 'act_K': 1.0, 'act_V': 0.96875, 'act_Z': 0.68359375, 'attn_pattern': 0.6144939661026001}, 'L2H6': {'act_Q': 1.0, 'act_K': 1.0, 'act_V': 0.9921875, 'act_Z': 0.9765625, 'attn_pattern': 0.9966692328453064}, 'L6H5': {'act_Q': 0.99609375, 'act_K': 0.99609375, 'act_V': 0.97265625, 'act_Z': 0.96484375, 'attn_pattern': 0.9947434067726135}, 'L4H6': {'act_Q': 0.99609375, 'act_K': 1.0, 'act_V': 0.98828125, 'act_Z': 0.9140625,

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer
Checkpoint: 141000 - Head Component Cosine Similarity to 143K: {'L8H9': {'act_Q': 0.984375, 'act_K': 1.0, 'act_V': 0.9765625, 'act_Z': 0.890625, 'attn_pattern': 0.9029083847999573}, 'L10H1': {'act_Q': 0.9609375, 'act_K': 0.99609375, 'act_V': 0.87109375, 'act_Z': 0.75390625, 'attn_pattern': 0.8369593620300293}, 'L4H11': {'act_Q': 0.99609375, 'act_K': 0.99609375, 'act_V': 0.98046875, 'act_Z': 0.984375, 'attn_pattern': 0.9971866607666016}, 'L6H6': {'act_Q': 0.9921875, 'act_K': 1.0, 'act_V': 0.984375, 'act_Z': 0.984375, 'attn_pattern': 0.9955798387527466}, 'L7H2': {'act_Q': 0.984375, 'act_K': 1.0, 'act_V': 0.96875, 'act_Z': 0.66796875, 'attn_pattern': 0.5947669744491577}, 'L2H6': {'act_Q': 1.0, 'act_K': 1.0, 'act_V': 0.9921875, 'act_Z': 0.9765625, 'attn_pattern': 0.996558427810669}, 'L6H5': {'act_Q': 0.9921875, 'act_K': 1.0, 'act_V': 0.9765625, 'act_Z': 0.96875, 'attn_pattern': 0.994422435760498}, 'L4H6': {'act_Q': 1.0, 'act_K': 1.

In [25]:
# load json
import pandas as pd
with open("similarity_through_time_last_pos.json", "r") as f:
    similarity_through_time = json.load(f)

def plot_head_component_cosine_similarity(head, similarity_through_time):
    # Convert nested dictionary to a list of dictionaries
    data_list = []
    for checkpoint, heads in similarity_through_time.items():
        for item, value in heads[head].items():
            data_list.append({'Checkpoint': checkpoint, 'Item': item, 'Value': value})

    # Convert list to Pandas DataFrame
    df = pd.DataFrame(data_list)

    # Use Plotly Express to create the line chart
    # Ensure each 'Item' gets its own line by specifying it in the 'color' parameter
    fig = px.line(df, x='Checkpoint', y='Value', color='Item',
                  title=f'Attention Head {head} Component Cosine Similarity Through Time')

    # Show the figure
    fig.show()

    return df

df = plot_head_component_cosine_similarity("L8H9", similarity_through_time)

In [26]:
# plot all heads
for head in similarity_through_time['1000'].keys():
    plot_head_component_cosine_similarity(head, similarity_through_time)

In [101]:
df.head()

,Checkpoint,Item,Value
0,1000,act_Q,0.265625
1,1000,act_K,0.142578
2,1000,act_V,0.039795
3,1000,act_Z,-0.009644
4,1000,attn_pattern,0.344754
